In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit,GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
from tqdm import tqdm


In [2]:

microbiome = pd.read_csv('../../data/raw/curated_metagenomics/relative_abundance.csv',index_col=0).transpose()
metadata = pd.read_csv('../../data/raw/curated_metagenomics/metadata.csv',index_col='sample_id',low_memory=False)

print(len(microbiome))
#get stool samples
metadata = metadata.loc[metadata.body_site == 'stool',:]

#Add obesity disease tags to disease BMI
to_change = metadata.BMI>=30
metadata.loc[to_change,'disease'] = 'obesity'

to_change = metadata.BMI<16
metadata.loc[to_change,'disease'] = 'severe_underweight'

# Remove all disease NaNs
metadata = metadata.loc[metadata.disease==metadata.disease,:]

#
to_keep = metadata.age_category != 'newborn'
metadata = metadata.loc[to_keep,:]

# Get the overlapping set of samples between metadata and microbiome data
overlapping_samples = list(set(metadata.index) & set(microbiome.index))
microbiome= microbiome.loc[overlapping_samples,:]
metadata = metadata.loc[overlapping_samples,:]

# target_metadata = metadata.loc[metadata.study_name == 'YachidaS_2019',:]
# target_microbiome = microbiome.loc[target_metadata.index,:]

20533


In [3]:
y = np.asarray( metadata.disease != 'healthy',dtype=int)
feature_names = microbiome.columns

In [4]:
np.sum(y)


6521

In [5]:
study_names = np.unique(metadata.study_name)

In [6]:
hmp_metadata = metadata.loc[metadata.study_name == 'HMP_2019_ibdmdb',:]
hmp_microbiome = microbiome.loc[hmp_metadata.index,:]
y_hmp = np.asarray( hmp_metadata.disease != 'healthy',dtype=int)
print(np.sum(y_hmp)/len(hmp_metadata))

yachida_metadata = metadata.loc[metadata.study_name == 'YachidaS_2019',:]
yachida_microbiome = microbiome.loc[yachida_metadata.index,:]
y_yachida = np.asarray( yachida_metadata.disease != 'healthy',dtype=int)
print(np.sum(y_yachida)/len(yachida_metadata))

qin_metadata = metadata.loc[metadata.study_name == 'QinN_2014',:]
qin_microbiome = microbiome.loc[qin_metadata.index,:]
y_qin = np.asarray( qin_metadata.disease != 'healthy',dtype=int)
print(np.sum(y_qin)/len(qin_metadata))

0.8063921327596804
0.6087662337662337
0.5189873417721519


In [10]:
studies,samples = np.unique(metadata.study_name,return_counts=True)

check_arr = np.transpose(np.asarray([studies,float(samples)]))